In [ ]:
%load_ext autoreload
%autoreload
import time, datetime
import os
import pandas as pd

pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
rootPath = "/content/drive/My Drive/주식과빅데이터/workspace"

Mounted at /content/drive


In [ ]:
def get_volumeKospiHistory(rootPath, code, name, start_date, end_date):        
    filePath = rootPath + "/Kospi/" + str(code).zfill(6) + "_" + name + ".csv"
    data = pd.read_csv(filePath, engine='python')
    
    datetime_index = pd.to_datetime(data['date'])
    df=data.set_index(datetime_index)
#     df.drop('date',axis=1,inplace=True)
    
    df = df.loc[start_date:end_date]
    
    return df

In [ ]:
def get_priceHistory(rootPath, code, name, start_date, end_date):
    filePath = rootPath + "/Price_Edit/" + str(code).zfill(6) + "_" + name + ".csv"
    data = pd.read_csv(filePath, engine='python')
    
    datetime_index = pd.to_datetime(data['date'])
    df=data.set_index(datetime_index)
#     df.drop('date',axis=1,inplace=True)
    
    df = df.loc[start_date:end_date]
    
    return df

In [ ]:
def get_volumeKosdaqHistory(rootPath, code, name, start_date, end_date):
    filePath = rootPath + "/Kosdaq/" + str(code).zfill(6) + "_" + name + ".csv"
    data = pd.read_csv(filePath, engine='python')
    
    datetime_index = pd.to_datetime(data['date'])
    df=data.set_index(datetime_index)
#     df.drop('date',axis=1,inplace=True)
    
    df = df.loc[start_date:end_date]
    
    return df

In [ ]:
code = "008500"
name = "일정실업"
start_date = '2010-01-01'
end_date = '2019-12-31'

def getKospiMergedTab(rootPath, code, name, start_date, end_date):
    vol = get_volumeKospiHistory(rootPath, code, name, start_date, end_date).fillna(0)
    p = get_priceHistory(rootPath, code,name, start_date, end_date).fillna(0)
    df = pd.concat([vol, p], axis=1, join='inner')
    
    return df

def getKosdaqMergedTab(rootPath, code, name, start_date, end_date):
    vol = get_volumeKosdaqHistory(rootPath, code, name, start_date, end_date).fillna(0)
    p = get_priceHistory(rootPath, code,name, start_date, end_date).fillna(0)
    df = pd.concat([vol, p], axis=1, join='inner')
    
    return df

df008500merged = getKospiMergedTab(rootPath, code, name, start_date, end_date)

In [ ]:
def accumConsecutiveCount(df):
    df['accum'] = (df['NetBuyAmount_Inst'] > 0).astype(int)
    last_index = df.index[0]
    init_index = last_index + pd.DateOffset(1)
    for index, row in df[init_index:].iterrows():
        if df.loc[index].accum > 0:
            df.loc[index,'accum'] = df.loc[last_index, 'accum'] + 1
        last_index = index
    return df

df008500 = accumConsecutiveCount(df008500merged)

In [ ]:
def computeRevenue(df):
    output_df = pd.DataFrame(columns=['매수일', '6th', '7th','8th','9th','10th','11th','12th','13th','14th','15th'])
    for i, _ in df[df['accum']>=5].iterrows():
        next_day = df.index.get_loc(i) + 1
        if (next_day+10) < df.shape[0]:                
            tmp_v = list()
            base_day = df.iloc[next_day-6].at['date'][0]
            tmp_v.append(base_day)
            base_price = df.iloc[next_day]['Open']
            for _ in range(10):
                close_price = df.iloc[next_day]['Close']
                revenue = ((close_price - base_price) * 100) / base_price
                tmp_v.append(revenue)
                next_day = next_day + 1
            tmp_series = pd.Series(tmp_v, index = output_df.columns)
            output_df = output_df.append(tmp_series, ignore_index=True)
    return output_df
# df = computeRevenue(df008500)

# file_path = rootPath + '/output/raw_' + code + "_" + name + ".csv"
# df.to_csv(file_path, index = False, header=True)

In [ ]:
def getSummary_max(code, name, df):
    output_df = pd.DataFrame(columns=['종목명','종목코드','6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
    tmpList = list()
    tmpList.append(code)
    tmpList.append(name)
    for i in range(6, 16, 1):
        df_min = df[str(i)+'th'].max()
        tmpList.append(df_min)
    return tmpList

def getSummary_min(code, name, df):
    output_df = pd.DataFrame(columns=['종목명','종목코드','6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
    tmpList = list()
    tmpList.append(code)
    tmpList.append(name)
    for i in range(6, 16, 1):
        df_max = df[str(i)+'th'].min()
        tmpList.append(df_max)
    return tmpList

def getSummary_avg(code, name, df):
    output_df = pd.DataFrame(columns=['종목명','종목코드','6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
    tmpList = list()
    tmpList.append(code)
    tmpList.append(name)
    for i in range(6, 16, 1):
        df_mean = df[str(i)+'th'].mean()
        tmpList.append(df_mean)
    return tmpList

def getSummary_med(code, name, df):
    output_df = pd.DataFrame(columns=['종목명','종목코드','6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
    tmpList = list()
    tmpList.append(code)
    tmpList.append(name)
    for i in range(6, 16, 1):
        df_median = df[str(i)+'th'].median()
        tmpList.append(df_median)
    return tmpList

def getSummary_prob(code, name, df):
    output_df = pd.DataFrame(columns=['종목명','종목코드','6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
    tmpList = list()
    tmpList.append(code)
    tmpList.append(name)
    for i in range(6, 16, 1):
        df_prob = df[df[str(i)+'th']>0].shape[0]/df.shape[0]
        tmpList.append(df_prob)
    return tmpList


In [ ]:
start_date = '2010-01-01'
end_date = '2019-12-31'

In [ ]:
min_output_df = pd.DataFrame(columns=['종목명', '종목코드', '6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
max_output_df = pd.DataFrame(columns=['종목명', '종목코드', '6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
avg_output_df = pd.DataFrame(columns=['종목명', '종목코드', '6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
med_output_df = pd.DataFrame(columns=['종목명', '종목코드', '6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
prob_output_df = pd.DataFrame(columns=['종목명', '종목코드', '6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])


file_list = os.listdir(rootPath+"/Kospi/")
kospi_file_list_csv = [file for file in file_list if file.endswith(".csv")]
total_day = len(kospi_file_list_csv)
total_process_time = 0
for i,v in enumerate(kospi_file_list_csv):
  try:
    code = v[:6]
    name = v[7:-4]
    start_time = time.time()
    dfMerged = getKospiMergedTab(rootPath, code, name, start_date, end_date)
    dfCount = accumConsecutiveCount(dfMerged)
    dfRevenue = computeRevenue(dfCount)
    if dfRevenue.shape[0] > 0:
      # file_path = rootPath + '/output/raw_' + code + "_" + name + ".csv"
      # dfRevenue.to_csv(file_path, index = False, header=True)
      tmp_series_max = pd.Series(getSummary_max(code, name, dfRevenue), index = max_output_df.columns)
      max_output_df = max_output_df.append(tmp_series_max, ignore_index=True)

      tmp_series_min = pd.Series(getSummary_min(code, name, dfRevenue), index = min_output_df.columns)
      min_output_df = min_output_df.append(tmp_series_min, ignore_index=True)

      tmp_series_avg = pd.Series(getSummary_avg(code, name, dfRevenue), index = avg_output_df.columns)
      avg_output_df = avg_output_df.append(tmp_series_avg, ignore_index=True)

      tmp_series_med = pd.Series(getSummary_med(code, name, dfRevenue), index = med_output_df.columns)
      med_output_df = med_output_df.append(tmp_series_med, ignore_index=True)

      tmp_series_prob = pd.Series(getSummary_prob(code, name, dfRevenue), index = prob_output_df.columns)
      prob_output_df = prob_output_df.append(tmp_series_prob, ignore_index=True)

    end_time = time.time()
    processed_time = int(end_time - start_time)
    total_process_time += processed_time
    print("{}/{} {}-{} Processed(처리시간:{}sec, 누적시간:{}sec) - 대상거래일수:{}, 5연속순매수일:{}".format(i+1, total_day, code, name, processed_time, total_process_time, dfMerged.shape[0], dfRevenue.shape[0]))
  except Exception as e: 
    print(e)
    continue    
    
file_path_max = rootPath + '/output/summary_5days_kospi_max.csv'
max_output_df.to_csv(file_path_max, index = False, header=True)

file_path_min = rootPath + '/output/summary_5days_kospi_min.csv'
min_output_df.to_csv(file_path_min, index = False, header=True)

file_path_avg = rootPath + '/output/summary_5days_kospi_avg.csv'
avg_output_df.to_csv(file_path_avg, index = False, header=True)

file_path_med = rootPath + '/output/summary_5days_kospi_med.csv'
med_output_df.to_csv(file_path_med, index = False, header=True)

file_path_prob = rootPath + '/output/summary_5days_kospi_prob.csv'
prob_output_df.to_csv(file_path_prob, index = False, header=True)

1/775 005930-삼성전자 Processed(처리시간:14sec, 누적시간:14sec) - 대상거래일수:2466, 5연속순매수일:230
2/775 000660-SK하이닉스 Processed(처리시간:14sec, 누적시간:28sec) - 대상거래일수:2466, 5연속순매수일:241
3/775 207940-삼성바이오로직스 Processed(처리시간:8sec, 누적시간:36sec) - 대상거래일수:2466, 5연속순매수일:86
4/775 051910-LG화학 Processed(처리시간:14sec, 누적시간:50sec) - 대상거래일수:2466, 5연속순매수일:230
5/775 035420-NAVER Processed(처리시간:16sec, 누적시간:66sec) - 대상거래일수:2466, 5연속순매수일:320
6/775 068270-셀트리온 Processed(처리시간:14sec, 누적시간:80sec) - 대상거래일수:2466, 5연속순매수일:216
7/775 005380-현대차 Processed(처리시간:14sec, 누적시간:94sec) - 대상거래일수:2466, 5연속순매수일:241
8/775 006400-삼성SDI Processed(처리시간:15sec, 누적시간:109sec) - 대상거래일수:2466, 5연속순매수일:286
9/775 035720-카카오 Processed(처리시간:17sec, 누적시간:126sec) - 대상거래일수:2466, 5연속순매수일:315
10/775 051900-LG생활건강 Processed(처리시간:16sec, 누적시간:142sec) - 대상거래일수:2466, 5연속순매수일:294
11/775 028260-삼성물산 Processed(처리시간:11sec, 누적시간:153sec) - 대상거래일수:2466, 5연속순매수일:187
12/775 036570-엔씨소프트 Processed(처리시간:16sec, 누적시간:169sec)

In [ ]:
min_output_df = pd.DataFrame(columns=['종목명', '종목코드', '6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
max_output_df = pd.DataFrame(columns=['종목명', '종목코드', '6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
avg_output_df = pd.DataFrame(columns=['종목명', '종목코드', '6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
med_output_df = pd.DataFrame(columns=['종목명', '종목코드', '6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])
prob_output_df = pd.DataFrame(columns=['종목명', '종목코드', '6일째 수익률', '7일째 수익률','8일째 수익률','9일째 수익률','10일째 수익률','11일째 수익률','12일째 수익률','13일째 수익률','14일째 수익률','15일째 수익률'])


file_list = os.listdir(rootPath+"/Kosdaq/")
kosdaq_file_list_csv = [file for file in file_list if file.endswith(".csv")]
total_day = len(kosdaq_file_list_csv)
total_process_time = 0
for i,v in enumerate(kosdaq_file_list_csv):
  try:
    code = v[:6]
    name = v[7:-4]
    start_time = time.time()
    dfMerged = getKosdaqMergedTab(rootPath, code, name, start_date, end_date)
    dfCount = accumConsecutiveCount(dfMerged)
    dfRevenue = computeRevenue(dfCount)
    if dfRevenue.shape[0] > 0:
      # file_path = rootPath + '/output/raw_' + code + "_" + name + ".csv"
      # dfRevenue.to_csv(file_path, index = False, header=True)
      tmp_series_max = pd.Series(getSummary_max(code, name, dfRevenue), index = max_output_df.columns)
      max_output_df = max_output_df.append(tmp_series_max, ignore_index=True)

      tmp_series_min = pd.Series(getSummary_min(code, name, dfRevenue), index = min_output_df.columns)
      min_output_df = min_output_df.append(tmp_series_min, ignore_index=True)

      tmp_series_avg = pd.Series(getSummary_avg(code, name, dfRevenue), index = avg_output_df.columns)
      avg_output_df = avg_output_df.append(tmp_series_avg, ignore_index=True)

      tmp_series_med = pd.Series(getSummary_med(code, name, dfRevenue), index = med_output_df.columns)
      med_output_df = med_output_df.append(tmp_series_med, ignore_index=True)

      tmp_series_prob = pd.Series(getSummary_prob(code, name, dfRevenue), index = prob_output_df.columns)
      prob_output_df = prob_output_df.append(tmp_series_prob, ignore_index=True)

    end_time = time.time()
    processed_time = int(end_time - start_time)
    total_process_time += processed_time
    print("{}/{} {}-{} Processed(처리시간:{}sec, 누적시간:{}sec) - 대상거래일수:{}, 5연속순매수일:{}".format(i+1, total_day, code, name, processed_time, total_process_time, dfMerged.shape[0], dfRevenue.shape[0]))
  except Exception as e: 
    print(e)
    continue    
    
file_path_max = rootPath + '/output/summary_5days_kosdaq_max.csv'
max_output_df.to_csv(file_path_max, index = False, header=True)

file_path_min = rootPath + '/output/summary_5days_kosdaq_min.csv'
min_output_df.to_csv(file_path_min, index = False, header=True)

file_path_avg = rootPath + '/output/summary_5days_kospi_avg.csv'
avg_output_df.to_csv(file_path_avg, index = False, header=True)

file_path_med = rootPath + '/output/summary_5days_kosdaq_med.csv'
med_output_df.to_csv(file_path_med, index = False, header=True)

file_path_prob = rootPath + '/output/summary_5days_kosdaq_prob.csv'
prob_output_df.to_csv(file_path_prob, index = False, header=True)